In [1]:
# -*- coding: utf-8 -*-
import os
import sys
import warnings

from glob import glob
import random
import time
import datetime
import re
import json
from tqdm import tqdm

import pandas as pd
import numpy as np

from google.cloud import bigquery

In [2]:
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_rows', 72)
pd.set_option('display.max_columns', 10)
pd.set_option('max_colwidth', -1)

In [4]:
def bigquery_to_pandas(query_string) :

    credential_path = '../99_credential/lawtalk-bigquery-2bfd97cf0729.json'
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path
    bqclient = bigquery.Client()
    
    b = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(
            # Optionally, explicitly request to use the BigQuery Storage API. As of
            # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
            # API is used by default.
            create_bqstorage_client=True,
        )
    )
    
    return b

In [6]:
query_string = """
WITH t1 AS (
    SELECT 
        _id
        , title
        , LENGTH(title) AS title_len
        , body
        , LENGTH(body) AS body_len
        , categories
        , REGEXP_EXTRACT(category, r'[a-zA-Z0-9]+') AS category
        , reject
        , viewCount
        , createdAt
    FROM `lawtalk-bigquery.raw.questions`
        , UNNEST(REGEXP_EXTRACT_ALL(categories, r'\\(\\'[a-zA-Z0-9]+\\'\\)')) AS category
)
, t2 AS (
    SELECT 
        t1.*
        , t2.name AS categoryName
    FROM t1
    JOIN `lawtalk-bigquery.raw.adcategories` AS t2
    ON (t1.category = t2._id)
)
SELECT * FROM t2
"""

In [7]:
df = bigquery_to_pandas(query_string)

In [75]:
reFunc1 = lambda x : re.sub(string = x, pattern = r"\n+|\t+|\{+|\}+|#+|\"+|\'+|\<+|\>+|\=+|\++|\-+|~+|\:+|\&+|\!+|\u2028+", repl = " ")
reFunc2 = lambda x : re.sub(string = x, pattern = r"\)+|\(+|-+|\.+|\s+", repl = " ")
reFunc3 = lambda x : re.sub(string = x, pattern = r"\s{2,}", repl = " ")

df.title = df.title.apply(reFunc1)
df.title = df.title.apply(reFunc2)
df.title = df.title.apply(reFunc3)

df.body = df.body.apply(reFunc1)
df.body = df.body.apply(reFunc2)
df.body = df.body.apply(reFunc3)

In [76]:
chk = df.groupby("_id").agg(
    title = ("title", np.unique),
    body = ("body", np.unique),
    categories = ("categoryName", list),
    cateCnt = ("categoryName", "count")
)


In [77]:
chk.cateCnt.value_counts()

1    71558
2    26071
3    10800
Name: cateCnt, dtype: int64

In [78]:
chk[chk.cateCnt > 1].assign(categories = lambda x : x.categories.astype(str)).groupby("categories").body.count().sort_values(ascending = False)

categories
['상법일반', '폭행/협박']               1338
['상법일반', '재산범죄']                1253
['상법일반', '명예훼손/모욕']             1169
['상법일반', '이혼']                  734 
['재산범죄', '채권추심']                671 
                               ...  
['보험', '상법일반', '이혼']            1   
['보험', '상법일반', '형사기타']          1   
['보험', '상속', '채권추심']            1   
['보험', '손해배상', '형사소송절차']        1   
['IT/테크', '계약일반', '명예훼손/모욕']    1   
Name: body, Length: 3139, dtype: int64

In [79]:
df = df[df.createdAt.dt.year >= 2020]

In [80]:
chk = df.groupby("_id").agg(
    title = ("title", np.unique),
    body = ("body", np.unique),
    categories = ("categoryName", list),
    cateCnt = ("categoryName", "count")
)


In [81]:
chk.cateCnt.value_counts()

1    47483
2    15392
3    9559 
Name: cateCnt, dtype: int64

In [114]:
chk[chk.cateCnt > 1].assign(categories = lambda x : x.categories.astype(str)).groupby("categories").body.count().sort_values(ascending = False).head(10).reset_index()

,categories,body
0,"['성범죄', '명예훼손/모욕']",483
1,"['건설/부동산', '임대차']",476
2,"['계약일반', '건설/부동산']",434
3,"['명예훼손/모욕', '폭행/협박']",428
4,"['계약일반', '임대차']",409
5,"['재산범죄', '손해배상']",406
6,"['재산범죄', '채권추심']",397
7,"['재산범죄', '대여금', '손해배상']",331
8,"['손해배상', '폭행/협박']",283
9,"['성범죄', '폭행/협박']",267


In [109]:
chk[(chk.cateCnt > 2) & (chk.categories.apply(lambda x : ('상법일반' in x) ))][["title", "body", "categories"]].sample(5)

,title,body,categories
_id,,,
61b7fac93179ee0021639a5a,음식 지적재산권 관련,"요약 기존에 있지않던 음식을 특허가 있지않은 상태에서 다른 가게에서 따라했다면 신고할 수 없는 부분인지, 그 가게에 똑같다고 댓글을 남겼다면 명예훼손이 성립하는지가 궁금합니다 1 A에서 개발한 토마토바질크림치즈 쿠키 / 토마토 바질 재료조합은 대중적이지만 쿠키이면서 딱 저런모양은 검색해도 나오지 않음 2 오늘 오픈한 B에서 A랑 너무 유사한 쿠키를 자기네 시그니처라고 홍보함 차이점은 토마토 바질 갈릭크림치즈라는 점 3 재료나 생김새가 너무 유사한데, 우리가 특허를 낸 것도 아니라서 검색해보니깐 음식 특허는 특이한 재료의 조합일때 가능하다고함 저 쿠키집에 대응할수 있는지 그냥 손쓸수가 없이 가만히 있을수밖에없는지 가 궁금합니다","[상법일반, 지식재산권, 명예훼손/모욕]"
600e9faedf9191001ebd524d,소음으로 시작된 다툼,"주택가에 세입자로 살고 있습니다 창문을 열면 손 닿을 거리에 옆집 판넬 벽과 지붕이 있습니다 옆집은 교회를 운영하고 있습니다 평소 음향기기로 찬송가 부르기, 판넬로 확장한 곳에서 음식 준비하며 울리는 소음 등으로 크게 불편을 겪었지만 이웃이고 하여 참고 살았습니다 며칠전 문 밖으로 나가서 하기 귀찮아 창밖으로 이불을 털다가 핸드폰을 그 집 지붕으로 2회 떨어뜨렸습니다 판넬이라 크게 울린듯 합니다 옆집 아저씨가 바로 집으로 찾아와 남의 집을 쳤나, 거구의 남성이 집을 부수는 줄 알고 심장이 떨린다며 항의해, 저도 순간 화가나 실수로 친거고 그건 죄송하다 근데 지금까지 교회 음향기기 소리까지 참으며 살았는데 그거 한번 실수로 쳤다고 찾아와서 이런식으로 따지냐, 이렇게 남의 집 문 두드리고 소리지르시는거 불법이라 항의했고 말싸움으로 번졌습니다 여자 혼자 있는 집에 남성이 찾아와 소리지르고 간 뒤로 마음이 너무 불편한데 증거가 없어 대처할 방법이 없습니다 앞으로 어떻게 대처하거나 증거를 수집해야 할까요","[환경, 상법일반, 임대차]"
6015c93db7b20b00821858d3,신용불량자 공증 차용 관련 형사 고발 가능성 문의,"아래 내용들에 대해 오프라인 상으로 법무 상담을 요청 드릴 예정입니다 채무자는 사업상 미팅 및 일시 투자 비용이 필요하다며 2020년 12월 20일 까지 35만원, 10만원, 100만원, 120만원으로 약 4차례에 거쳐서 비용을 빌려갔음 금액을 빌려갈 때에는 채무자는 항상 채권자에게 웃돈을 주며 갚겠다고 약속을 하고, 확실한 금액까지 채권자에게 알려주었음 2021년 01월 31일까지 총 60만원 변제 되었음 2020년 12월 20일, 투자금을 확보하기 위해 선금 비용이 필요하다며 1200만원을 채권자에게 요청하여, 채무자에게 송금하였음 이 때, 채권자는 채무자에게 2020년 12월 30일까지 지금까지 빌린 금액까지 포함하여 2200만원으로 변제 하기로 약속하였으며, 이에 대한 공증을 한 공증 사무소에서 진행하였음 이외 법인회사 설립 시 지분 및 차량 지급 등을 메신저로 약속하였음 채무자는 이를 기일 내에 변제 하지 않았으며, 약 17회에 거쳐 변제 기일을 미룬 바 있음 이 때, 초기에는 채권자의 동의가 있었으나, 이후 채권자의 동의도 없이 변제 기일을 미루었음 신용 정보 회사에 채무자에 대한 신용 정보 조사를 의뢰하였고, 채무자는 다른 개인, 업체들에서도 큰 금액을 빌리고 변제 하지 않은 것이 확인되었음 신용 불량인 상태에서 채권자에게 돈을 빌려간 것으로 확인하였음 채권자는 투자금 확보를 목적으로 채무자에게 비용을 빌려준 것이기 때문에, 본인이 빌려준 금액에 대하여 사용 용도 및 이체 내역을 채무자에게 요청하였으나, 채무자는 투자 비용으로 빌려간 것이라고 하였고, 투자 역시 현금으로만 거래를 진행하였다는 말만 반복한 후, 이체 및 출금 내역을 채권자에게 전달하지 않았음 1 채권자는 채무자를 사기 및 기망 혹은 타 법률 위반 으로 형사 고발이 가능한지? 2 채권자가 실제로 채무자에게 빌려준 금액은 1450만원인데, 채무자가 2200만원으로 갚는다고 하여 공증을 2200만원으로 하였는데, 이를 받으면 높은 이자율로 문제가 되는지?","[계약일반, 상법일반, 재산범죄]"
5fb07e0d27a1420223107b3d,쇼핑몰 사업 양도 관련 분쟁입니다,"안녕하세요 몇달전 쇼핑몰을 양도 받았는데 사업이 가품을 판매하는 쇼핑몰 입니다 상표법 위반인데 이런경우에도 양도자한테 민사소송이 가능한지 궁금합니다 해당 사업을 양도 받을때 양도자는 본인이 운영하는 사업, 상점과 모든 거래처를 다 저한테 넘겨준다는 계약, 자신은 이제 이쪽 사업을 아예 하지 않을거라고 했습니다 재차 우려되는 부분이라 여러번 물어봤었고 계약서특약 사항에도 작성해 뒀습니다 갑은 사업 양도후 해당 사업으로 어떠한 이익도 얻을수 없고 제3자에게 컨설팅 또한 할 수 없다 근데 몇달뒤 우연히 양도자가 운영하는 듯한 상점을 찾아냈고 상품내용이나 거래처가 동일 했고, 계좌번호가 모두 양도자 명의로 되어 있었습니다 무려 5개나요 그래서 직접 찾아가서 이야기를 했습니다 사업에 대한 모든 권리를 나한테 양도 했는데 왜 몰래 뒤에서 사업을 다시 하고 있냐 그러니 하는 말이 이건 본인이 운영하는게 아니라 친누나가 운영하는거라고 말을 돌리더군요 돈은 본인 통장으로 받으면서요 이렇게 어처구니 없는 대답하니 제가 할 말이 없더군요 양도자 컴퓨터를 뒤질수도 없고 통장을 까볼수도 없으니깐요 상점 개설일을 보면 이미 저에게 양도 전에 만들어 둔것과 양도 후 만들어둔것 다양하게 있었습니다 여러개의 상점중 저에게 1개만 양도하고 사업 전부를 양도 한다는듯 말했던거죠 애초에 계약 위반이니 해당 계약 사항은 무효 아닌가 생각이 듭니다 혹시 소송으로 가능성, 리스크가 감당 가능하면 변호사님과 상담 후 빠른 진행하고 싶습니다 긴글 읽어주셔서 감사합니다","[계약일반, 상법일반, 재산범죄]"
60255c87b102a1009c9a7ac8,탈세신고가 들어올거같습니다,"면세 사업장을 운영중입니다 앙심를 품은, 전영업직원이 탈세신고를 하겠다고 협박을 하는 중입니다 탈세는 저희가 A라는 매입처에서 매달 500만원치 매입을 합니다 그리고 저희 B라는 매출처에 매달 500만원치 매출을 하는데요 저희 A 매입처에서 매입시 매입계산서 발행증을 저희가 받지 않고 B 매입한걸로 하여 발행해주었습니다 저희는 500만원치 매입계산서도 받지않고 매출계산서도 발행해주지않은 셈이된거죠 이렇게 1년에 4800만원정도를 해왔어요 승계받을때부터 그렇게 해오길래 그렇게 하는줄 알고 하다가 최근에야 잘못된것을 알고 바로잡아 제대로 하는중입니다 승계전엔 현금으로 주고받았었고, 승계받은후 근 4년동안 현금으로 매입하였거나 불규칙적이게 입금한 내역이 있습니다 지금도 경영난에 너무 힘들어 폐업을 할까말까 고민중인데 이런타격까지온다면 너무 힘듭니다 1 지금이라도 자진납세를 한다면, 많이 감액되나요? 2 대락적인 벌금은 어느정도가 될까요 ? 6% 구간입니다 3 탈세신고라는게 신고한다는것만으로도 조사가 시작되는건가요? 어떤 요건이 충족되어야 탈세신고가 되나요?","[세금, 상법일반, 재산범죄]"


In [102]:
chk[(chk.cateCnt > 2) & (chk.categories.apply(lambda x : ('성범죄' in x) & ('명예훼손/모욕' in x)))][["title", "body", "categories"]].sample(5)

,title,body,categories
_id,,,
600d46c57fd30f0091e72ea0,"성인과 만13세와 연애, 그리고 제3자의 욕설 및 협박","안녕하세요 저는 올해 24살 만 22세 이고 여자친구는 올해 15살 만13세 입니다 2021년 1월초에 친구의 소개로 서로 연락하면서 호감이 생겨서 사귀기로 했습니다 처음에는 여자친구의 아는 언니때문에 연애하는것을 부모님에게 숨기고 몰래 연락하면서 연애를 하다가 약 1월 15일경에 부모님이 알아채시고, 여자친구가 아끼는 부모님 친구분 이하 삼촌 이 있는데 그분이 저희 연애를 반대하더라구요 처음에는 여자친구 부모님은 저의 사귀는건 허락하셨어요 근데 삼촌이라는사람이 저한테 욕설과 미성년자 성추행, 강간 혐의로 경찰에 신고하겠다고, 여자친구랑 연락하지마라, 만나지마라, 헤어져라 이런소리를 하더군요 여자친구랑 연애를 하면서 스킨쉽은 서로 좋아서 성관계 전까지 키스까지 했습니다 인터넷 찾아보니까 만13세 이상이면 서로 호감이있어서 스킨쉽을 진행한경우 처벌되지 않는다고 알고있습니다 이게 사실인지 궁금합니다 다만, 저는 강제로 스킨쉽을 진행하지 않았고, 서로 합의하에 스킨쉽을 진행했다는 통화녹음본 이 존재하고, 여자친구의 친구들도 제가 강제로 진행하지 않았다고 증인들도 있습니다 여기서 제가 궁금한건 여자친구랑 아무런 상관없는 제3자 삼촌 부모님친구분 이 저한테 욕설 문자메세지로 경찰에 신고하겠다는등, 통화를 통한 욕설 녹음본이있습니다 또 뒤에서 여자친구한테 제욕을 직접적으로 했다고 하더라군요 대화내용 캡쳐본있음 또한 여자친구의 친구들 및 선배 언니 및 오빠 들에게도 제가 여자친구를 강제로 강간하고 성추행하려고 했다고 이야기하고 다녔습니다 대화내용 캡처본 있음 이 경우 제가 이 삼촌이라는 사람을 명예훼손, 모욕죄, 협박,허위사실유포 혐의로 고소가 가능할까요?? 그리고 만약 삼촌이라는 사람이 사람이 제가 여자친구를 강간 성추행 했다고 고소를 할경우 명예훼손, 모욕죄, 협박,허위사실유포 무고죄까지 포함해서 고소가 가능한지 궁금합니다","[성범죄, 명예훼손/모욕, 폭행/협박]"
5e36f6c0995e0001bd68dbba,안녕하세요 라인에서 성폭행 고소가능성,안녕하세요 데이트어플을 통해 어떤분을 알게되었고 서로직접 만난적고 이름도 모르는 상태였는데 제가 라인으로 대화하자고해서 상대방도 응하였습니다 서로 음란사진이 아닌 각자의 사진을 보내주었고 야한이야기를 서로 하기시작했습니다 저는 가슴좀 보여달라고 하였고 상대방은 거절하였고 상대방도 저에게 야한이야기를 하였슺니다 그러던중 상대방여성이 내일그럼 성관계를 하자고 하였는데 저도 좋다고 말을 하였고 십분정도 곰곰히 생각해보니 위험한것같아서 죄송하다고 거절을 하였습니다 이러한상황인데 혹시 상대방이 저를 고소하거나 신고할수있나요? 라인은 탈퇴하였습니다 두려운게 혹시나 이를 빌미로 저를 신고하거나 상대방이 할까봐 두렵네요 전 죄송합니다 못만날것같아요 좋은하루되세요하고 겁이나 그냥 탈퇴를 하였습니다 저상황중 제가 고소 당하거나 벌금을 물상황이 있는지 불안해 남깁니다,"[성범죄, 형사기타, 명예훼손/모욕]"
61da6931ee856d006c8f5535,롤 1ㄷ1 욕설 채팅,어느 사람이 친추 걸어서 받았더니 못하면 하지말라고 민폐라며 시비를 걸더라고요 처음에는 너나 이러다가 계속 욕설을 유도하나 티어 어디냐고 겜 삭제하라 하길래 화나서 넌 줘도 안 먹고 안 가진다고 네 부모나 삭제하라 했더니 고소한다 말하고 친삭하고 갔네요 롤 1ㄷ1 채팅으로 주고 받은 대화이고 그분의 특정 이름이나 사는 지역 전화번호도 아예 모르고 닉도 딱히 언급해서 욕설한 건 아닙니다 제가 알기론 1ㄷ1 대화 욕설은 특정성 성립이 안되어서 고소가 불가능한 걸로 알지만 제가 줘도 안 먹는다 이 발언으로 성적 ? 관련으로 고소가 가능한가요 ? 그 외에 이분이 고소할 수 있는 부분이 있을까요 ? 먼저 친추 걸고 시비를 걸어서 욕설을 하였던 부분이고 자랑할 거리는 아니지만 혹시나하여 여쭈어봅니다,"[성범죄, 수사, 명예훼손/모욕]"
5f028af7f322c002dfc0ffc9,직장 상사의 고백 성희롱 성립 가능할까요?,"약 1년 전, 유부남 직장 상사가 회식자리에서 저를 따로 불러 고백을 했습니다 자꾸 꿈에 너가 나온다, 사회가 아닌 학교에서 만났으면 너를 따라다녔을 것 같다, 예쁘다 등등 당시 사내에 바로 신고하고 싶었지만, 직장 내에서 불이익을 당할까 걱정이 되어 신고를 하지 못했습니다 현재 이직을 준비하고 있는데, 퇴사할 때 성희롱으로 고소를 하고 싶습니다 그러나 cctv 영상, 녹취 파일 등의 증거가 없어 걱정입니다 사건 직후 화가 나서 지인에게 보낸 카톡 내용, 회식 자리에 참석하였을 때 저를 따로 불러내는 것을 본 직원의 증언 정도가 전부입니다 이럴 경우 성희롱 성립이 어려울까요?","[노동/인사, 성범죄, 명예훼손/모욕]"
5e31ad96c2a06607b5326f79,만약 고소 당한다면 어느 형벌로 어떤 피해를 받게 되나요?,"안녕하세요 올해로 16살이 된 중학생입니다 지난 23일에 저는 피파 온라인4라는 게임을 하다가 만난 상대한테 경기가 너무 안풀려서 너희 엄마 보x 냄새가 아직까지 나네,내가 너희 엄마 강간해버릴꺼야 등의 각종 패드립과 성드립을 난무하였습니다 물론 상대방도 가정교육을 못 받았네라는 말을 하긴 했습니다 경기가 끝나갈 때쯤 이거 다 스샷 해놨다고 고소를 한다고 하네요 그래서 경기가 끝난 후 친추를 걸었고 저는 나이를 성인이라고 속이고 상대방에게 이거 신고 안된다고 우기며 진짜 하지 않았어야 했는데 저희 아버지가 경찰청장이라고 하였습니다 그리고 의대에 합격했다는 것까지 고소를 당할까봐 두려운 마음에 거짓말을 했습니다 너무 무서워서 사과를 했지만 상대방은 무시하고 게임을 껐습니다 그리고 3일이 지난후 게임에 들어가보니 상대방이 아직 연락이 안왔냐고 합니다 이 연락이 무슨 연락일까요 그리고 제가 아무것도 안왔는데 무슨 연락이냐고 물어보니까 무슨 연락인지는 안 가르쳐주고 설 연휴 끝나고 연락 갈거라고 합니다 저는 무서운 마음에 다시 정중히 사과를 했습니다 하지만 그 와중에도 저는 끝까지 나이를 속였습니다 계속 상대방이 나이를 물어봤기 때문이었죠 제가 계속 나이 질문을 회피하니 아 됐고 너희 부모님 연락처 알아내서 패드립 하고 다니지 말라고 말만 하면 나는 됨이라고 말하고 친구를 삭제 당하고 연락이 안되는 상태입니다 어제부터 계속 원래 나이를 밝히고 사과를 다시 할려고 기다려도 상대방은 친추를 받지 않고 있습니다 저는 이 행동을 너무 반성하고 있고 며칠동안 인터넷을 뒤져보며 게임 내 채팅으로 인해 고소된 사례가 어떤 것들이 있는지를 찾아보며 잠도 못자고 불안한 상태입니다 또한 제가 나름 학군이 좋은 지역의 학교에서 전교 1% 이내의 내신 성적을 가지고 있고 현재 자사고 진학을 목표로 해서 최종적으로 의대에 들어가는게 꿈입니다 혹시 지장이 있을 수도 있기 때문에 제 미래가 두렵습니다 고소말고 부모님께 전화만 오면 좋을텐데요","[체포/구속, 성범죄, 명예훼손/모욕]"


In [104]:
chk[(chk.cateCnt == 1) & (chk.categories.apply(lambda x : ('성범죄' in x)))][["title", "body", "categories"]].sample(5)

,title,body,categories
_id,,,
5eba4ae0782d3400d947ae50,음성 랜덤채팅 어플에서 테러로 신고당할 수 있나요?,익명인데 목소리로 랜덤채팅하는 어플이 있습니다 따로 회원가입 절차는 없고 들어가면 임의로 만들어진 닉네임으로 활동하게 됩니다 제가 오늘 거기서 불특정다수에게 정력에 락스가 좋다던데 이런식으로 보냈는데 그 메시지를 받은 상대방이 테러로 신고한다고 했습니다 어떤 죄명으로 신고가 되는지 또 신고가 되면 처벌을 받는지 좀 알려주세요ㅠㅠ,[성범죄]
5fe89f56eaff3400e3f4ad44,디씨인사이드 야짤 유포,제가 기분이 좋아 디씨인사이드라는 사이트에 예전에 다운 받은 야짤을 올렸습니다 사진은 모르는 여성분이였고 얼굴은 가려져 있었지만 가슴 골반까지 노출이 되어있고 골반부터는 스타킹을 신어서 성기노출은 없었습니다 고닉으로 디씨에 가입되어잇는 상태였고 약 20초가량 뒤에 게시글을 지웠지만 누군가가 이미 신고 넣었다고 말하였습니다 겁을 먹어서 회원 탈퇴까지 하였습니다 이런한 경우 실제 경찰서에서 연락이 올 경우와 처벌을 받을 경우는 어떻게 될까요? 나이는 02년생으로 1달뒤면 청소년 보호법에서 벗어나고 2021년 10월에 민법상 성인이 됩니다,[성범죄]
5ee84c70b9be0500fe81d291,카카오톡 메신저로 성희롱 문자를 받았습니다,"어느날부터 최근 3개월 이내 전혀 모르는 사람이고 신원도 특정할 수 없는 사람에게 몸매가 예쁘다, 목소리 들어보고 싶다는 둥 카카오톡 메신저와 보이스톡 요청이 한달 간격 정도로 간헐적으로 오기 시작했습니다 처음에는 그냥 무시하고 말았는데요, 오늘 아침에는 기어이 성희롱을 담고 있는 문자를 받았습니다 아래는 받은 카톡 내용의 전문입니다 안녕 검정색 망사스타킹 의자에 앉아있는 사진 정말 최고 어디사는지요? 만날수있나요? 그대 너무예쁘고 몸매보니 못참겠는데 X질좀하고X물좀 그대 X구녕에 싸고 입에도 해줄께 X해줘요 누군지도 모르는 사람에게 더이상 이런 수치스런 카톡과 보이스톡 받는 것은 싫습니다 이제는 확실하게 조치를 취해야 겠다는 생각이 들어서 자문을 구하려고 글을 남겼는데요 질문 1 고소나 형사입건 쪽으로 마음 먹었으면 제가 더 준비해야할 증거라던가 할게 있을까요? 유도신문을 통하여 성희롱의 증거를 더 확보하는 것은 부적절할까요? 질문 2 변호사님께서는 이 사건 같은 경우 어떻게 조치를 취하는게 제일 낫다고 보시는지요?",[성범죄]
5f19077bc5fa6901d4cf7328,성희롱으로 경찰조사를 받았습니다,"길에서 여성에게 성희롱 발언 형의로 경찰조사를 받았습니다 경찰은 도망가는 cctv영상과 여성이 녹음한 듯한 10초 정도 음성 파일 틀어 들려주었습니다 40분 정도 조사를 했고 질문은 최종학력/월수입/집 자가 유무 였으며 술주량과 그날 얼마나 마셨나 정도였습니다 그날 마신 술의 양은 주량의 2배 정도로 진술했습니다 영상과 음성을 들려주고 인정하느냐? 영상과 음성 들으니까 어떤 기분이 드느냐 정도가 큰 질문이었습니다 술 때문에 기억은 안나지만 증거가 있으니 제가 실수를 한게 맞는거 같다고 진술했고, 충격적이라고 대답을 했습니다 경찰이 조사후 사건자체는 경미한 범죄라 법원으로 자료를 보내면 아마 겸범죄로 벌금이 나올거라고 했습니다 행여 술 더마시고 터치라도 했으면 큰일 났을뻔한 사건이라고 술좀 줄이라고 말했구요 법원에 서류 보낼 때 문자 줄테니 집에 서류 못오게 하는건 알아서 하라고 했습니다 궁금한 점은 첫번째 경찰말로는 경미한 사건이고 벌금 받을거 같다고 했는데 전과 남냐고 했더니 전과는 평생 남는다고 경찰이 말하더군요 경범죄는 2년까지만 기록이 남는걸로 알고 있는데 경찰자체적으로는 자료를 계속 가지고 있다는 소린가요? 다른 개념인지? 제가 벌금을 받고 난 후 진술서를 기반으로 여성이 경찰에 제 정보를 요청해서 다시 민사로 저를 소송할 수 있나요? 경찰 말 중에 그 여성분 사진은 안보여드릴께요 만나게 되면 얼굴 보게되니 인지 만나서 얼굴 보게 될테니까요 같은 말을 한 것 같아 불안하네요 그때는 인정을 했지만 변호사를 선임해야 겠죠? 두번째 법원 같은 곳에서 벌금 통지서? 같은게 날아올 때 본인만 받을 수 있는 등기로 오나요? 직접 가서 발송전에 미리 받을 수 있지와 내용에 성희롱 이런 단어가 적혀 오는지 궁금합니다",[성범죄]
5e9917db5f020d02157e1127,토렌트로 몰카물 아무생각없이 다운받았고 5분만에 놀라서 삭제했습니다,안녕하세요 위에 말그대로 한달전에 토렌트로 음란물을받다가 파일에 몰카라는게 적혀있는걸 보아 바로 깜짝놀라 보지도않고 바로 삭제하였습니다 한달전이어서 기억도 잘안나지만 5분도안되는 시간이었던것 같습니다 찾아보니 토렌트는 업로드까지 자동으로 되는 시스템이라는대 한순간의 실수로 한달간 밥도제대로못먹고 걱정 근심만 늘어가는것 같습니다 어떻게 되는걸까요? 잘못되면 취업제한까지 갈수도 있다는 글을 보아서 하루하루 힘이 드내요,[성범죄]


In [105]:
train_ = df[df._id.isin(df.groupby("_id").categoryName.nunique()[df.groupby("_id").categoryName.nunique() == 1].index)]

In [107]:
train_[["title", "body", "categoryName"]]

,title,body,categoryName
6,여러명의 자녀가 부모한테 증여하는 경우,자녀 세명이서 5천만원씩 어머니에게 증여하는경우 어머니께서 증여세를 얼마나 내는지 산출방식이 궁금합니다 ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////,증여
7,민사소송 가능할까요?,"앉녕하세요 다름이 아니라 민사소송 관련해서 여쭐게 있어 상담글을 남깁니다 우선 핵심요지만 말씀드리면, 이웃간에 담배꽁초 무단투기로 인한 다툼니다 저는 단독주택에 거주하고 있고, 저희 집 뒷편으로 담 하나를 사이에 두고 약 20 30세대의 규모의 빌라가 있습니다 올해 봄 저희집 내부 뒷공간?에 담배꽁초가 몇개 떨어져있는걸 보게 되었습니다 저희 집 뒷편은 집에서 사용하는 20kg짜리 LPG가스통이 2 3개 늘 있습니다 즉, 절대 담배꽁초가 투기되면 안되는 공간입니다 저희집은 흡연자가 한명도 없기에, 조금 의아했지만, 담배갑이 아닌 꽁초 몇개였기에 그냥 치우고 말았습니다 근데 며칠 뒤 다시 집 뒷쪽에 가스문제로 한달에 1 2번 갑니다 갈일이 있어 가보니 이번에 지난번보다 더 많은 꽁초와 담배갑에 라이타 까지 있는겁니다 같은종류의 담배갑만 8개 떨어져 있는걸 봤을때 같은 사람이 반복적으로 투기한다고 생각했고, 워낙 위험한 공간 LPG가스가 항상 있기에 폭발 위험이 있음 이라 이건 큰 일나기 전에 반드시 잡아야겠다 생각해서 약 9만원 상당의 홈cctv를 설치했고 2만원주고 sd카드까지 넣어서 확인한 결과 옆 빌라 특정호수에서 던지는걸 확인했고, 바로 가서 항의했고, 직접 와서 담배꽁초와 담배갑을 치우고 Cctv와 sd칩 가격을 달라고 했는데 사과와 배상은 무슨 조낸 뻔뻔하게 나오는겁니다 누구는 가스통때문에 몇달을 불안에 떨었는데요 이샛기가 그래서 너무 화가 나 인생은 실전이라는걸 알려주려고 민사소송을 걸려고 합니다 옆집 때문에 쓴 돈 12만원 그 동안 시달린 오죽했으면 cctv를 설치했을까요 정신적위자료,소송비용 등을 청구하려고 하는데 이걸로 증거로 소송이 가능한지 하단참고 또 가능하면 청구비용은 어느정도가 적정선일까요? 증거자료 투기된 담배꽁초와 담배갑 사진 투기하는 장면이 찍힌 동영상 사람 식별가능",환경
8,치아교정 부작용에 대해 보상받고자 하는데 어떻게 받을 수 있을까요?,"2012년 9월부터 A치과에서 교정치료를 받다가, 2014년 3월에 A치과 상호명이 B치과로 바뀌고 원장도 a에서 b로 바뀌었습니다 이후 2014년 10월 경에 교정장치를 빼고유지장치를 부착한 뒤 1년 간격으로 관리를 받으러 방문하고 있습니다 그런데 몇 년 전부터 잇몸 퇴축이 발생해서 외관상으로도 좋지 않고 관리기 어려우며 통증도 발생했습니다 그래서 얼마 전에 큰 병원 방문해 치료를 받으려 상담을 진행하는데 퇴축 원인이 해당 치아 뿌리가 앞쪽으로 나와있어서 그렇다고 합니다 게다가 치은이식 위해 부분교정 진행하는 안내를 하시면서 부작용을 알려주시는데, 윗니가 교정 과정에서 뿌리가 동그랗게 마모되었다고 하셔서 깜짝 놀랐습니다 교정 시작하먼서부터 이러한 부작용들에 대해 전혀 안내받지 못했고 이후 방문했을 때에도 b원장은 이러한 설명을 전혀 하지 않았기 때문입니다 매번 방문할 때마다 큰 변형 없다고 말하였는데, 병원에서 위와 같은 말을 듣고 보니 왠지 치열도 고르지 않은 느낌이라 신경이 계속 쓰입니다 어쨌든 치은 퇴축 치료를 위해 부분교정 및 치은 이식술 등을 시행해야 한다고 하는데 이 과정의 치료비와 위자료, 그리고 교정 비용 일부를 병원으로부터 받아내고 싶습니다 어떻게 받아야 할지 알려주시면 감사하겠습니다",의료
14,개인정보보호법 상담 요청,"안녕하세요 1 어떤 주장에 동의하고자 개인의 이름과 주소, 서명이 기재된 연대서명 자료가 기재된 이름, 주소가 실제와 일치하는지 여부는 알 수 없음 개인정보보호법의 적용을 받는지 여부 2 연대서명 자료를 범죄 신고 고소 또는 고발 의 목적으로 이용하는 경우에 개인정보보호법 위반인지 여부 등에 대해 전화 상담을 받고 싶습니다 개인정보보호법 전문 변호사님 연락 부탁합니다",개인정보
17,구상권소송 해야할지 부당이득반환신청을 해야할지 모르겠어요,"제 명의로 전여친 부모님집에 안마의자를 렌탈받았습니다 사이가 안좋아지면서 헤어졌고, 안마의자는 전여친이 달달이 돈내면서 쓰겠다하고, 계좌변경까지 했는데, 1 2회만 납부하고 3년내내 미납했습니다 제명의 이기에 제가 완납을했습니다 전여친부모님은 3년내내 사용하고나서 이제는 제꺼 가져가라 합니다 이경우 부당이득반환신청 해야 맞는건가요? 너무 억울합니다 증거자료는 계약서 설치장소 ,통장내역 있습니다",부당이득
...,...,...,...
152165,현역 군인인데 전애인에게 협박죄로 고소당했습니다,"입대 후에 관계가 깊었던 애인과 술 문제로 갈등이 지속적으로 생기고, 결국에는 입대 3달 뒤에 이별을 통지 받았습니다 하지만 연락을 지속적으로 하다가, 첫휴가 때 만나서 얘기를 하자고 했고, 결국 만나서 다시 잘됬습니다 하지만 부대에 복귀를 하니, 전애인이 태도가 바뀌고 결국 다시 이별하게 됬습니다 이 사건 이후에 서로 감정의 골이 깊어지고, 여러 사건 때문에 서로 진흙탕 싸움을 이어갔습니다 이 과정에서는 협박성 연락은 없었습니다 그러다 모종의 사건으로, 전애인과 다툰 뒤 폭로할 것이다 라고 협박하진 않았습니다 제가 전애인의 새어머니께 전애인의 술관련 사건들을 솔직하게 말씀드렸습니다 이 행동을 한 이유는 흥분한 감정도 있었지만, 술관련 문제를 새어머니께서는 전혀 모르시고, 전애인이 전혀 술 조절을 하지 못하기 때문이었습니다 그리고 2달 뒤에 다시한번 전애인과 모종의 사건으로 다시한번 다투게 됬습니다 이때도 폭로할 것이다다 라고 협박성 연락을 취하진 않았습니다 그때 제가 저번에 새어머니께 말씀 차마 못드렸던 전애인 예민한 술관련 사건을 말씀드렸습니다 이 연락과정에서 제가 전애인에게 너 다시 행복하지 못하게 해줄께 , 지금 남친이 너 이런거 알고도 너 좋아하나 보자 라는 식의 추상적인 협박성 카톡을 보냈습니다 이 경우에 제가 죄가 성립될 가능성이 높은지, 그리고 수사를 받을 때 어떤 식으로 방어를 해야될지 여쭤보고 싶습니다",폭행/협박
152171,전혀 기억나지 않은 전날 만취상태에서 제가 119구급대원 뺨을 1대 때렸다고 하네요,"거래처 사장님 한분과 10일전 pm 6시에 1차 저녁을 먹으며 소맥 술을 했고 pm 7시에 2차 자리를 옮겨 또 술을 했고 pm 9시에 3차 제 사무실로 옮겨 12시까지 계속 술을 먹었습니다 2명이서 먹었으니 방역법 위반은 아닌 것 같습니다 장장 6시간 동안 술을 먹었습니다 할 얘기가 많았나 봅니다 사무실에서 술 마신 기억은 있는데 아침에 눈을 뜨니 집이더군요 필름이 끊겨서 전혀 기억이 없습니다 문제는 지금 부터입니다 아침에 일어나니 오른쪽 눈썹이 찢어져 있길래 알아보니 집에 오는 길에 넘어져서 다쳤나봅니다 근처 CCTV 담당 관제센터에서 이를 확인하고 119구급차를 보냈나 봅니다 가까운 병원응급센터까지는 잘 갔다고 하는데, 차에서 내려 구급대원 한명이 저를 붙잡아서 강제로 응급실로 이송하려 했고 제가 치료를 거부하고 도망 가려고 했나봅니다 아마도 만취상태에서 방어본능이 발동되어 납치되나 생각이 돼 도망가려 했나봅니다 오늘 형사한테 전화가 왔는데 그 과정에서 제가 119구급대원 뺨을 1대 때렸다고 그래서 소방법 제9조 위반 구급대원폭행으로 형사입건대상이라고 전화가 왔습니다 CCTV 증거 자료도 있다고 합니다 정말 자다가 봉창이라는 말이 이럴 때 쓰는 말인 것 같습니다 전혀 기억도 없고, 그날 급히 연락받고 데리러 온 조카도 119구급대원 뺨을 1대 때렸다는 얘기를 다른 대원에게 전해 듣고 그냥 못가니 신원보증을 하고 왔답니다 이런 상황에서 어떻게 해야하나요? 경찰서에서 오라는데, 1차 조서작성이 중요할 것 같은데, 어떻게 대처해야 할지 답답합니다",폭행/협박
152173,특수상해 폭행죄관련,어제 친구한명이 인터넷으로

In [108]:
train_[train_.categoryName == "상법일반"][["title", "body", "categoryName"]]

,title,body,categoryName
34866,아파트 동대표와 선관위 해촉에 관해 여쭤봅니다,"안녕하세요 저는 대구의 한 공공임대 아파트의 입주민 입니다 저희 아파트는 20년에 첫 입주를 하게된 아파트 입니다 이곳엔 입주 2년 전부터 입주 예정자들의 커뮤니티로 사용된 인터넷 카페 가 있습니다 네이밍 변경을 위해 수백여명의 주민들이 가입을 하여 현재까지 소통을 하고 있는 공간입니다 참고로 이곳은 개인정보를 본인이 적지 않으면 노출이 되지 않는 카페입니다 선관위에서는 동대표 선거당시 이곳이 아무도 동대표 선거에 대해 특정후보를 거론 된 적이 없음에도 불구하고 이곳에서 선거활동을 하는 것은 불법이라며 사찰을 하려 하였고, 동대표 선출 조건에 입주민 카페 가입금지, 임기기간 중 역시 입주민 카페 가입금지 조항을 걸어 놓았습니다 몇몇 입주민은 구청에 문의하여 선거관리위원회가 관리규약을 위반하였다는 답을 얻었으나 자체적으로 협의하란 말만 하였고, 선관위는 이를 무시하였습니다 이후 선출된 동대표들은 입주민들과 어떠한 소통을 하려하지 않았습니다 해결해야 할 문제들이 많았음에도 6개월이 지난 현재까지 어떠한 문제조차 해결이 된 것이 없었습니다 현재 동대표와 선거관리위원회 분들은 송주법 주민대표 자리까지 겸직하여, 아파트 내부 조경을 엎고 새로운 시설을 만들자는 얼토당토 않는 의견을 내고 있습니다 또한 선관위에서 정한 입주민 카페 가입금지 조항을 없애지 않았음에도 불구하고, 현재 수백여명이 넘게 모여있는 입주민 카페가 동대표가 자리에 없으니 비공식 카페다 며 카페를 관리하시는 분께 카페를 넘겨라고 요구하였고, 매니저분이 거절하자 다음날 아침 아파트 공식 카페를 개설하겠다고 아파트 내 공지사항을 올리고, 주민 동의도 없는 현수막까지 내 걸었습니다 이 정도 추진력었다면 전 날 카페를 넘기라는 말도 요청도 요구가 아닌 답이 정해져있는 통보였습니다 현재 입주민들은 동대표라는 사람들이 입주민들을 말을 들어주려 하지 않고, 오히려 군림하려 하는 상황에 분통을 터트리고 있습니다 이들을 해촉하기 위한 방법을 변호사님께 여쭙니다",상법일반
34885,렌트카 계약관련 질문좀 드리겠습니다,"2020년 10월경 캐피탈사를 끼고 렌트사에 보증금이 아닌 선수금을 지급, 일정금액 지원받는 방식으로 계약을 했고 현재 3개월째 이용중입니다 첫 달에 지원금 지급이 늦어졌는데 신생렌트사라 시스템이 아직 갖추어지지않아 그렇겠거니 생각했습니다 하지만 둘째달도 지원금 지급이 늦어져 확인을 해보니 코로나로 인한 매출이익의 감소로 지원금 지급이 늦어지고있으니 조금만 기다려달라는 문자가왔고 얼마 후 렌트사와 지원금에 관련하여 협약을하고있는 회사에서 지원금을 미정산해주어서 지원금지급에 어려움이 있어 시간을 조금 더 달라는 문자를 받았습니다 애초 계약했던 금액의 3배이상되는 금액이 납부되다보니 금전적인 부담을 느끼고있는 상황이고 이 상황이 언제까지 지속될지 모르는터라 걱정도 많이 됩니다 이런 불안한 상황에서 계약을 해지하게 된다면 렌트사의 계약이행 위반으로 선수금을 돌려받을 수 있을까요? 추가로 렌트사에사 받은 확약서와 캐피탈사에서 받은 계약서를 가지고 있습니다",상법일반
34887,등기 사고 관련해서 질문드립니다,"직계가족이 제 초본과 인감증명서, 인감 도장 을 가지고 제 동의없이 할 수 있는 일이 있나요?? 이런 상황에 자주 일어나는 사고가 어떤 것이 있나요 위험할까요? 참고로 가족과 친하지 않습니다 제가 아는 이유는, 직계 가족이 제 명의를 이용해 법인 설립시 이사로 등기하려고 한다고 까지 들었습니다 이 부분 확실하지 않으나 관련해서 일어날 수 있는 사고가 있을까요??",상법일반
34895,손님이 물건을 교환해달라고 요청하는데 법적으로 교환절차가 어떻게되는지 궁금합니다,안녕하세요 물건을 판매하는 소매업 소상공인입니다 2월 15일 한 손님이 물건을 사가셨습니다 2월 16일 물건에 하자가 있는것 같다며 물건을 가져오시고 교환해달라고 요청했습니다 물건은 의료기기 전자기계제품입니다 이때 저는 소매업자로서 물건을 사입해서 판매하는 입장이니 물건에 하자가 있다면 물건을 판매하는 회사에 교환을 신청하는게 맞다 고 설명드렸습니다 물건에는 소비자상담실 전화번호도 적혀있습니다 손님은 물건을 팔았으면 교환도 해주는게 맞지않냐 고 하십니다 법적으로 소매업자에게 물건교환의 책임 또는 의무가 있는지 궁금합니다,상법일반
34909,생후4개월 아기 예방접종 관련,"로타 바이러스 예방접종종류는 2가지가있는데 로타텍은3차까접종 로타릭스는 2차까지 접종으로 저희아기는 로타텍3차접종까지 하는걸로 1차를 접종했습니다 근데 병원에서 아기수첩에 로타릭스2차접종하는걸로 잘 못 적어놓고 미국에 접종한거 기록해놓는 사이트에는 로타텍3차까지 접종하는걸로 잘 기록해놓고 로타바이러스 2차 백신을 맞으러 갔는데 간호사나 의사나 아기수첩에적혀있는걸로만 확인하고 사이트에있는걸로 확인안하고 그냥 접종을 하였습니다 원래 아기수첩은 확실한 서류가 아니어서 사이트 들어가서 확인을 해야된다고함 그래서 1차와 2차접종 백신이 다른걸로 접종하게되었습니다 그래서 의사가 2주뒤에 다시와서 로타텍으로 다시 접종하면되고, 4차까지 백신을 맞아도 무리없다라고 알려줬는데 제가 찝찝해서 2일지나고 병원에 다시 문의를 해보니 의사가 처음에는 2주뒤에 맞아도 상관없다라고 했지만 몇분뒤에 다시 전화오더니 다시알아봣더니 최소 4주가 지나야지만 재접종이 가능하고 로타텍으로 1차첩종을 맞은거면 4차까지 맞으면안되고 3차에서 끝내야하되는거였습니다 이런 잘못된 정보로 진단하고 설명하고 접종시킨 무능력한 의사가있다는거에 너무 화가나네요 지금은 이상증상이 없지만 커가면서 이상증세가 나타날까봐 걱정되고 그 의사가 원망스럽습니다 어떻게하면 좋을까요",상법일반
...,...,...,...
36495,"중개보조원 계약진행 , 공인중개사법 위반","중개보조원 자격으로 1년전 전세계약을 직접 진행하다가 공인중개사법 위반으로 형사고소를 당한 상태입니다 상대방은 변호사와 함께 고소를 한 상태이며 , 개업공인중개사,보조중개원 둘다 벌금이 있는걸로 알고있습니다 고소인이 형사고소할때 증거로 전화 음성녹음을 제출하였으며 , 그 녹음안엔 보조중개원이 계약진행했다고 되어 있습니다 저희쪽에서 조치할수 있는 부분에 대해 자세한 상담을 받고싶습니다",상법일반
36498,"동업을 하였지만, 사업자명의가 저로 되어있는데 상대방이 법적으로 주장할 수 있나요?","2018년 10월 a가 전 남자친구 미용실을 차려주겠다는 권유를 받고 동업을 하게 되었습니다 계약서 작성을 하지 않았으며, 저는 3천만원 a는 1억을 투자하여 창업을 하였습니다 국가자격증이 있어야 미용실을 차릴 수 있었기에 사업자 명의, 부동산 계약, 거의 모든 가입자명의는 저로 되어있습니다 동업을 하면서 저는 월급을 받았으며 모든 수익금 그리고 경영에 일체 관여하지 않았습니다 2019년 12월 이별을 하게되었고, 2020년 2월 4월 3개월 a 월 급여 550만원을 지급하고, 순수익으로 8 a 2 나 나눴습니다 15개월가량 모든 수익금 대략 5000만원 이상 을 가져갔고, 개인 지출도 사업자카드로 사용하여, 임의 측정 후 퇴사 시에는 5000만원을 주었습니다 현재는 a가 미용실을 자기거라고 주장하고 있는데 직접적으로 본인의 미용실이라고 주장할 수 있나요? 주장을 할수 있다면 예상되는 근거가 뭐가 있을까요? 임의 측정에서 과측정되어있다면 청구 진행이 가능할까요?",상법일반
36504,친구의 권유로 마약? 을 한 상황,카카오 오픈 채팅방에서 만난 친구를 2020년 10월경 만나서 오픈 채팅방 애들과 함께 술을 마시고 친해지는 관계를 가졌습니다 회사 입사교육 전날 강남에 잘 곳이 없어서 그 친구에게 부탁을 하여 그 친구 집에 가족분들이 있음에도 불구하고 저를 재워줬습니다 그래서 저는 그 친구에 대한 믿음이 더 있었습니다 그리고 시간이 지나고 2021년 3월 7일 오후 8시경에 그 친구에게 뭐 하냐고 연락이 왔었고 게임을 하고 있다고 연락을 준 뒤 그 친구가 바람을 쐬러 가자고 저의 주소를 물어봐서 저는 가르쳐줬습니다 오후 9시 30분경 그 친구가 어디 있냐고 해서 집에 있다고 그러니까 나와라고 해서 나갔습니다 그리고 차에 탑승하여 여의도로 가는 도중에 

In [110]:
train_.categoryName.nunique()

70

In [111]:
train_.categoryName.value_counts().sort_values(ascending = False).index

Index(['성범죄', '명예훼손/모욕', '재산범죄', '폭행/협박', '임대차', '이혼', '손해배상', '계약일반', '상속',
       '형사기타', '노동/인사', '교통사고/범죄', '채권추심', '건설/부동산', '형사소송절차', '기타법률서비스',
       '민사소송절차', '지식재산권', '민사기타', '대여금', '가사기타', '회생/파산', '위증/무고', '상법일반',
       '의료', '행정', '민사집행', '도박', '수사', '주거침입', '개인정보', 'IT/테크', '마약/대마',
       '등기/등록', '보험', '매매', '공정거래', '기업일반', '국방/병역', '아동/소년범죄', '문서위조', '환경',
       '약식명령/즉결심판', '가압류/가처분', '금융', '세금', '부당이득', '조합/사단', '증여', '지급명령',
       '소비자/집단소송', '군형법', '친족', '이민/비자', '식품/의약', '조정/화해/중재', '엔터테인먼트', '헌법',
       '성년후견', '국제', '유공자/보훈', '경범죄', '도급/용역', '공증', '체포/구속', '언론/방송', '대국가범죄',
       '기본권', '뇌물', '지역'],
      dtype='object')

In [112]:
train_.categoryName.value_counts().sort_values(ascending = False)[:6]

성범죄        8980
명예훼손/모욕    7225
재산범죄       6412
폭행/협박      2875
임대차        2780
이혼         2420
Name: categoryName, dtype: int64

유료상담 카테고리 빈도수와 비교

In [39]:
pd.concat([train_[train_.categoryName == y[0]].sample(5) if y[1] > 5 else train_[train_.categoryName == y[0]] for x, y in train_.categoryName.value_counts().sort_values(ascending = False)[6:].reset_index().iterrows() ])[["title", "body", "categoryName"]].to_csv("chk/chk.csv", index = False)

In [15]:
train_.categoryName.value_counts().sort_values(ascending = False).reset_index()

,index,categoryName
0,성범죄,10320
1,명예훼손/모욕,9817
2,재산범죄,9530
3,이혼,5009
4,폭행/협박,4996
5,임대차,4606
6,상법일반,4109
7,채권추심,2894
8,상속,2154
9,손해배상,1929


one = []
two = []
three = []

In [22]:
train_[train_.categoryName == "계약일반"][["title", "body", "categoryName"]]

,title,body,categoryName
12221,이럴 땐 어떻게 해야되는지요?,"안녕하세요 카페 컨설팅을 하고있는 사람 입니다 간략하게 요약하자면 약 한달전, 카페 오픈 컨설팅 계약을 체결하였습니다 그 후, 인테리어 공사 중 인테리어 지연을 이유로 하지 않겠다고 의뢰인이 주장하고 있습니다 계약서에는 계약 해지 시, 요청자가 일천만원 위약금이 명시되어있고 귀책사유로 인한 해지시 이천만원 위약금이 명시되어있습니다 인테리어 지연도, 언제까지 끝내겠다는 것도 없었으며 서로 도의적으로 소통하며 합의하였는데 그 이후로 말을 바꿔서 주장하고 있어서 순순한 뜻으로 받아들이지 못하고 있습니다 어떻게 해야 되는지요?",계약일반
12228,차용증상 차용인과 돈을지급받은 통장사실이다른경우,"개인채무입니다 차용증을 작성하였고, 차용인은 김** 보증인으로는 처 ,아버지,어머니 그리고 담보로 차용인 의 아파트와 보증인 아버지의 집을 담보로 일금 5천만원을 차용한사실이있습니다 헌데 차용인은 김**이었으나 입금된통장은 처인 허** 앞으로입금되었으며 입금자도 전부 다릅니다 보증인은 김**을 보증한사실을 인정하나 처인 ***을 보증하지않았기에 본차용증은 무효하다라고 말씀하십니다 차용인 인 저또한 현재 차용증은 무요하다라고 주장할수있을까요 처 이자 보증인 허**은 본인앞으로들어왔으니 차용증상에 담보물이 무효화 된다면 처앞으로된 채무들모두 회생 절차를 밟아 처리하고자 합니다 처앞으로 채무가많고 연체도 되있는상황입니다 이번에 모두를 넣어서 회생코자 합니다 차용증상 담보물건이 제외된다면 좋을것 같습니다 변호사님 답변 부탁드립니다",계약일반
12234,건축이 계속 미뤄지고 있는 분양받은 건물 해지할 수 있나요?,"작년에 건물을 짓기 위해 건축을 하는 업체와 계약을 했습니다 처음부터 게약서를 작성할 때 제 계약 날짜랑 입주일날짜가 달라서 다시 쓰자 했는데 상관없다고 해서 그냥 진행했습니다 그리고 중도금 2차 지불날짜는 건축시 지불한다고 적혀있었습니다 무튼 그래서 총 4천만원 중 중도금 1차 2500만원을 입금하고 나머지는 건축할 때 입금하기로 했는데, 그 뒤로 분양팀장이랑 분양이사가 연락두절이 되었습니다 그래서 건축사무소를 통해 조합장 연락처를 받아서 연락을 했더니, 기존 건물들이 준공이 안나서, 그것이 완료되면 짓자고 하며 기다려 달라고 했습니다 해지는 언제든지 해주고 원금은 1주일 후 정도에 입금가능하다고 했습니다 그래서 일단 기다리고 있었는데 조합장한테 연락이 안와서 해지를 요청하니, 해지를 하면 계약금 손해가 있다고 했습니다 제가 계약위반에 대한 위약금을 받아야할 것 같은데 손해라뇨 법적으로 계약해지가 가능한지, 위약금을 받을 수 있는지 문의드립니다",계약일반
12236,안녕하세요 조합원 관련 내용에 대해서 문의하고싶습니다,"2017년 5월19일 창립조합원으로서 계약금을 2000천만원 가량 냈습니다 아직 조합원 인가를 받지 못한 상황이구요 계약당시에는 토지 계약률이 91퍼센트라고 광고한 상황이었는데, 결국에는 조합추진위원회에서 약속했었던 토지 매입에 실패하였다는 말과 함께 토지 면적과 세대수를 1400세대에서 750세대로 줄이겠다고 일방적으로 통보받았습니다 저는 조합추진위원회가 계약할 당시의 내용을 어길 뿐만 아니라 3개월간 홍보관 문을 닫고 대행사가 계속해서 바뀌는등 계약 당시 내용과 반대로 일방적으로 일을 처리해온터라 신뢰감이 바닥이 났고 결국 조합원 탈퇴를 요구했습니다 그쪽에서는 확인서를 발급해주면서 먼저 1000만원은 돌려준 상황이고 나머지 1500만원은 조합원 인가가 나면 돌려주겠다고 하였습니다 그러나 저는 이제 조합원에 소속되어있지도 않고 일방적으로 계약 내용과 다르게 일을 추진한 조합원때문에 탈퇴하게된 상황이기때문에 조합원 인가가 날때까지 기다려야할 의무가 없다고 생각합니다 또한 조합원 인가가 계속해서 나지 않을 경우의 나머지 돈의 반환에 대해서는 알려주지 않고 있는데 이런경우 어떻게하면 나머지 돈을 빠른 시일 안에 돌려받을수 있을까요?",계약일반
12237,전자책 출판 계약해지 관련 문의를 드립니다,저는 장르 소설 작가로 전자책 전문출판사와 A라는 작품에 대해 지난 2월 집필 전에 제목만 정해진 상태로 출판계약을 맺었습니다 집필의 효율성을 높이기 위해 N사 웹소설에 연재를 하기 시작했는데 뜻밖에 인기를 얻어 정식 계약을 앞두고 있습니다 이 경우 제가 A작품의 제목을 바꿔 N사와 계약하고 출판사에는 다른 작품을 A라는 제목으로 줘도 법적인 문제가 없는지 궁금합니다 만약 불가능하다면 어떻게 대응해야하는지 이러한 경우에 출판사의 작가에 대한 계약해지 손해배상 청구 범위가 어느정도 될지도 궁금합니다 변호사님 조언을 부탁드립니다,계약일반
...,...,...,...
18590,동업자가 계약위반을 하여 손해를 보고 있는데 법적으로 어떻게 받을수 있나요?,"제가 비용을 투자하고 동업자는 운영을 맡아서 하기로 동업계약서는 작성하여 19년 3월 부터 노래방을 시작하였습니다 사업자 등록증은 제 명의로 되어 있고요 동업자가 운영하면서 매출장부를 매일 저에게 보내왔고, 저는 동업자가 운영을 잘할 것으로 믿고 운영에 대해 관여를 안했습니다 그런데 영업개시일 2 3개월 후부터 약속한 매월 매출이익을 자꾸 미루어 주기 시작하였고, 이상하게 여겨 장부를 확인하였으나, 장부의 내용도 틀리는 경우도 많고, 매출액 중 개인적으로 사용하는 내용도 있는거 같아서 수차례 요청하고 또 요청하였습니다 하지만 동업자가 계속 이런저런 이유로 해결되지 않고 있다가 최근 몇 개월 전부터 아애 돈이 입금조차 안되고 있고, 심지어 월세까지 밀리고 있는 상황입니다 동업자가 임의로 최근 일주일 이상 영업도 전혀 안하고 있고, 전화조차 피하고 있습니다 법적 조치를 취하여 손해배상과 책임을 묻고 싶은데 어떻게 해야할까요?",계약일반
18595,계약마감기한을 못지켰더니 자해를 요구합니다,개인 간의 계약입니다 신청자에게 선금을 받고 특정 기한까지 작업물을 완성해주기로 했는데 기한 내에 마감을 하지 못하여 상대방이 SNS상에 저에 대한 내용을 올렸습니다 이에 저는 신청자에게 연락해 사과하며 환불과 작업물을 완성해주겠다고 다시 말했으나 상대방은 필요 없으니 본인의 눈앞에서 손에 상처를 내 혈서를 작성할 것을 요구하였습니다 계약서와 불이행시 사항은 작성하지 않았고 메신저 대화로 주고받은 내용입니다 기한 내에 작업물을 완성하지 못한 상황은 제 잘못이지만 계약을 불이행한 건으로 환불이나 위약금 대신 제 몸에 스스로 상처를 내라고 요구하는 부분에는 문제가 없는지 궁금합니다,계약일반
18596,앱개발 외주업체와 계약 기한 지나고 오류투성인 결과물 소송으로 계약금 환불 가능할까요?,"모바일 앱 창업을 위해 외주업체를 선정하였고 계약을 하였습니다 선금 2400만원 지급 하였고 검수 후 1600만원 지급 계약입니다 계약상 기한 마감은 10월 6일 이였으나 마감일 2주 앞두고 개발이 지연된다는 답변을 들었습니다 이에 정상적인 제품으로만 만들어달라는 답변을 하였고 이부분이 암묵적 동의가 되나 걱정됩니다 현재 12월 중순이 지나가고 있으나 받은 결과물은 오류 투성이입니다 회원가입조차 되지 않고 기본 기능들에서도 오류가 가득하여 도저히 납품을 받을 수가 없는 상태입니다 이거 너무 심한거 아니냐 , 담당자는 이걸 테스트 해본 것이냐 따지니 도리어 담당자는 당신이고 본인들은 시키는걸 하는 관계라고 말을 합니다 그러면서 문제가 무엇이냐 말하면 다 수정해주겠다고 말하는데 이미 몇차례 수정 작업 요청을 한 사항들도 개선이 되지 않고 있습니다 더 기대하고 기다리는게 너무 힘이 들어 계약 파기를 희망하는데 소송하여 환불 가능할까요?",계약일반
18597,매도자 일방적 계약파기 통보,"부동산 매매계약을 3달전 진행하였고 이는 중도금없이 계약금 10%와 10월말 잔금일에 잔금을 지급하는 계약을 하였습니다 10월이 되어 잔금 지급 한달 전 매수인은 의무를 이행하기 위하여 대출심사를 신청하였습니다 또한, 현재 거주중인 집의 임대인에게 계약 해지를 통보하여 임대계약의 만료를 요청하였습니다 그리고 2주 후 심사결과 발표 하루전 매도인에게 계약을 파기한다며 배액배상 받을 계좌번호 요구 문자를 받았습니다 이 경우 문자에 응하지 않고 대출이 승인된 후 실

In [17]:
train_[train_.categoryName == "상법일반"][["title", "body", "categoryName"]].sample(10)

,title,body,categoryName
36736,주권 발행 전에 주식을 양도받고 명의개서 했습니다,"안녕하세요 제가 회사의 주권 발행이 있기 전에 주식을 양도받고 명의개서를 받았습니다 나중에 알아보니, 주권 발행이 있기 전이고 회사 설립일이나 신주납입일로부터 6개월이 지나지 않은 경우에는 주식 양도를 받아도 무효라고 하더라고요 제가 이 경우에 해당되는데요 그런데 어찌저찌 명의개서는 된 상황입니다 그렇다면, 어쨌든 명의개서는 되었으니까 제가 회사에 주권 행사를 할 수 있는 건가요? 아니면 명의개서가 제대로 된 게 아니니까 주권 행사도 못 하는 건가요?",상법일반
36629,대표이사와 주주간의 갈등,"2년된 법인 대표이사인데 주식을 30% 소유하고 있습니다 그리고 주주 갑이 40%, 주주 을이 30% 소유하고 있습니다 경영악화로 폐업의 위기에 있는데 경영에 참여하지 않은 주주들이 본인의 자본금을 돌려달라고 하는 중입니다 배당을 한적은 없음 개인 돈으로는 줄 여력이 없어서 법인돈을 대표이사가 출금하여 주주 투자금을 반환하면 어떤 문제가 생기는지 궁금합니다 투자금을 반환함과 동시에 다른 명의의 주주를 구할 예정입니다 과점주주 방지를 위해",상법일반
41441,음란물 유포혐의로 벌금형500만원 으로 기소돼었습니다 벌금생각치도않게 많아서,안녕하세요 저는올해75년생 미혼남자입니다 일단 개요는 이렇습니다 파일공유사이트 토렌트 아닙니다 에 일본 모자이크처리된 야동을 대략250개 가량을 올렸다가 경찰조사후 2017년11월30일부로 벌금500만원에 약식기소한다는 문자가왔습니다 이사이트가 회원가입후 성인본인 인증을 하지않으면 다운자체가 불가능하고 일본에서 정식 제작되어진 모자이크처리된것들이라 불법이란사실을 모르고 올렸습니다 실제로 이렇게해서 포인트로 제가 얻은이득은 60만원 정도인듯합니다 제가 무지해서 불법이란걸 몰랐던어쨋건 법을 위반했다니 거기엔 의의를 제기할생각은 없습니다 다만 동종전과도 없는 초범인데 생각치도 않게 위반사항에 비해 너무과한 벌금액수라 너무 당황스럽습니다 아직 약식기소 상태이고 약식명령은 아직안떨어졌지만 판사님들 업무량에 볼때 대부분 검사님이 기소한대로 나온다고 들었습니다 현재 80을 바라보시는 노부모님 두분과 미혼인 상태로 제가 모시고 살고있습니다 제가 몸도 좋지않아서 3 4일에 한번씩 주기적으로 병원치료를 받고있는 만성질환 환자이기도하구요 제명의로 된 제산은 하나도 없습니다 경제적으로 500만원은 낼 능력이 도저히 안돼서 나온다면 결국 노역장유치가 확정적인데 만성질환으로 노역장 생활 또한 힘들것같습니다 질문1 벌금을 좀낮추어 300만원이하는 봉사활동으로 대신 할수있다고 알고있는데 사정하면 가능할런지요? 질문2 2018년1월7일 부터 500만원이하 벌금형도 집행유예 제도가 시행된다는데 약식명령후에 정식재판청구할생각인데 새로시행되는 제도적용이 저도 가능할런지요 결혼도 안하고 혼자살다보니 취미로 가볍게 즐기던게 법죄행위라니 너무 당황스럽고 답답합니다 하루하루 잠도 잘못자고 스트레스로 밥도못먹고있습니다 상담부탁드립니다 감사합니다,상법일반
45205,상표법위반 합의 불발시 어떻게 되나요ㅜ,인터넷에서 의류사이트를 운영중입니다 얼마전 계좌송금이 안되길래 은행에 알아보니 가압류신청이 되어있네요 명품st제품 판매로 인해 합의를 진행해야 한다고하는데 판매매출은 약 300 순수익은 150만원 정도입니다 판매서류를 모두 제출하고 기다리다보니 오늘전화가 왔는데 합의금을 1500만원을 요구하네요 정말 다지불해야하는건지 지불하지않을경우 민사소송을 진행한다고 얘기하네요 명품회사에서 그돈으로 합의를 원한다는 내역을 이메일로 보내달라고 하니 그걸 자기네가 보여줄 의무는 없다고 합니다 합의가 안되서 나중에 민사소송이 진행되고 판사가 보길 원하면 그때 볼수있다고 얘기하구요 지금 가압류 3000 을 걸어놨는데 소송시에는 5000으로 올린다고 얘기하는데 그건 무슨소리인지 잘모르겠습니다 가압류를 해제할방법은 없는지 그리고 합의금을 모두 지불해야 하는것이 맞는지 궁금합니다 또 합의가 불발될경우 소송에서 지게되면 5000만원을 내야하는건가요?,상법일반
37265,민물 양식장인줄 모르고 낚시를 하였는데 주인이 합의금 3천만을 요구합니다,"12시경 낚시를 좋아하는 저는 낚시를 하러가다 급한 용무 생리현상 때문에 차에서 내려 일을보고 갈려다 앞에 보이는 둠벙 같은곳에서 낚시를 하였습니다 울타리나 양식장표시도 없고 해서 낚시를 하였다가 가물치 양식장인 주인이와서 낚시하는 모습과 가물치 한마리잡고있는 모습을 사진찍어 갔구요 저때문에 양식하는 물고기들이 밥을 못먹고 스트레스로 죽어간다고 합의금3천만원 이라는 거액을 달라고합니다 안주면 신고한다고 하는데 어떻게 해야할지 모르겠습니다 ,합의를 봐서 물어주어야 하나요?",상법일반
37816,"매도인과 공인중개사의 잘못으로 인한 손해배상청구, 어떻게 하나요?","상가를 매입하였습니다 임대료가 200만원에 부가세 별도라고해서 상가를 매입하였으나, 몇 달 후 알고보니 임대료는 부가세 포함해서 200만원이었습니다 제가 간이과세자여서 부가세를 받지 않는다고 생각하여 부가세 제외한 200만원만 받으며 몇달이 지났습니다 임대차계약서는 공인중개사와 매도인의 말만 믿고 그대로 승계했습니다 확인해보니 매도인은 작정하고 저를 속였고, 공인중개사는 부가세 언급이 없는 임대차계약서는 부가세 포함으로 보는것인데 아예 반대로 알고 있는 등 상가 거래에는 초보였습니다 제 상가는 임대료도 임대료지만 건물가치에 큰 타격을 입었습니다 200만원에 부가세 별도인 상가가 순식간에 182만원에 부가세 별도인 상가가 되어버렸습니다 그래서 손해배상청구를 하고 싶습니다 그런데 이 경우, 매도인과 공인중개사의 책임 비율이 어떻게 되나요? 만약 50 50이고 내가 청구하고 싶은 손해배상액이 1000만원이면 매도인 500만원 공인중개사 500만원씩 청구하면 되는건지요 그리고 한 소송장에 피고1, 피고2 이런식으로 매도인과 공인중개사를 동시에 소송 가능한지 알고 싶습니다",상법일반
40382,동거남에게 금전요구 및 협박을 당하고 있습니다 어떻게 대응해야 하나요?,"오십후반이시고, 이혼상태이신 이모분이 계십니다 새로운 분을 만나셔서 동거를 한지 2년이상이 되었는데 나중에 듣고 보니 이모가 가진 얼마안되는 재산 서울아파트한채 을 노린것 처럼 지속적인 금품요구를 하고 있는 상태로 확인되었습니다 최근에는 개인사정이 어려워졌는지 금품요구와 함께 협박도 심하게 하고 있는 상태라, 이모가 신변의 위협을 느끼고 있는 상태입니다 몇가지 증거자료 협박문자, 금품요구, 경찰신고 는 있는 상태로 확인되며, 최근 조카인 저에게 고민을 털어놓으셨고 이에 따른 법적인 대응 방법을 상담받고 싶습니다 이모명의의 아파트에 동거중인상태고, 동거남이 집에 있는 상태라, 이모는 현재 집에 들어가지 못하고 모처에서 생활하고 계십니다 변호사를 통해야 하는지, 아니면 경찰을 통해야 하는지 , 이문제를 해결하기 위한 여러가지 접근법을 알고 싶습니다",상법일반
37411,제가 쓴 각서가 효력이 있나요?,지금 같은브랜드 미용실에서 4년6개월동안 일을 하고있습니다 그런데 회사에서 대학교를 같이 보내주는 계약학과라는 전형이있는데 직원들이 대학교만 졸업하고 회사를 퇴사한다는 이유로 저에게 학교를 졸업한 후에도 3년동안 일을 더하겠다는 각서를 자필로 적으라고 해서 이름에 사인 까지 했습니다 나중에 졸업하고 그만두면 제가 쓴 각서가 효력이있나요? 각서 내용은 이 매장에서 근무한지 4년6개월이 되었습니다 ㅇㅇㅇ원장님과 많은 분들의 도움으로 여기까지 오게되었습니다 앞으로 더열심히 하자는 의미에서 학교 졸업후에도 퇴사하지않고 3년이상 성실하게 근무하겠습니다 이렇게 썼습니다,상법일반
41004,담임선생님을 형사소송하고 싶습니다 좀 급해서요 ㅠㅠ,"심장병이 있는 학생이 담임선생님의 권유, 압박으로 안정을 취하지 못하고 학교 내에서 야간 자습을 하던 도중 학생이 쓰러졌습니다 그 때 감독선생님이시던 담임선생님은 학교 내에서 일을 키우기 싫어 119를 부르지 않고 아는 의사에게 전화를 걸어 응급처치법을 물어보았습니다 그 의사가 말 한대로 응급처치 약인 설하정을 투여하였고 CPR을 실시하였으나 CPR교육을 제대로 받지 않은 담임선생님은 잘못된 방법

In [15]:
train_[train_.categoryName == "상속"][["title", "body", "categoryName"]].head()

,title,body,categoryName
46660,아버지와 고모들이 합의한 할머니의 상속분할결정을 제가 소송을 걸수있나요?,"할머니가 얼마전에 돌아가셔서 제 아버지와 형제들이 상속분할을 논의하고있는데요, 땅 분할결정에 제가 너무 화가나서 소송을 하려고합니다 지금 70평인 땅을 장남인 우리아버지는 5평 지금현재 제 큰언니, 즉 장녀가 운영하는 가게의 평수 만 갖고 나머지 65평을 작은아버지의 큰아들에게 주기로 한 것입니다 원래 아버지와 작은아버지가 땅을 반반씩 나누는게 맞는건데, 손녀손자들한테가기까지 상속세를 2번이나 내야하니 바로 손녀손자들에게 주자고 합의했고, 그 과정에서 저희집은 아들이없으니 작은아버지집 큰아들에게 65평이나 주어야한다는 어른들의 이상한 결론이 나온겁니다 큰아들이 제사를 할것이니 더 주어야된다구요 평당 500만원이라 65평이면 3억 2천5백만원입니다 저희 큰언니는 2천5백밖에 못받구요 지금 이렇게 확정된건 아니지만 저희 아버지 성격이 자기몫을 못 챙기는 성격이라 고모들 사탕발림에 넘어가고있는 중입니다 고모들은 자기들은 이득도손해도보는게없지만 제사지내게될 큰아들이 불쌍하다며 큰아들에게몰아주자고하고있구요 만약 정말 이렇게 땅을 나누게 된다면 손녀인 제가 상속소송을 할수있나요? 승소가능성은 있나요? 수임료와 승소비, 소송비는 어떻게되나요? 저에게 떨어지는게 하나도없어도 되니 아버지가땅을 반 가졌으면 좋겠습니다 아버지가 장남이라 거의3년동안 치매걸린할머니 간병했구요, 저쪽집안은 1년? 6개월? 정도밖에 간병하지않았어요 제사 제가 지내도 되니까 땅 반을 꼭 저희집안 몫으로 하고싶습니다 빠른 답변 부탁드립니다",상속
46662,대출금 상환 및 상속포기에 대해,"A는 국민연금 10년 미만 가입자로 반환일시금을 받을 때까지 몇 년 남음 그 외에 가진 재산 없음 부모님은 사망 미혼으로 배우자와 자녀 없음 친형제는 4명 현재 월세 원룸 거주 그러나 원룸의 계약은 A가 아닌 A의 조카 명의로 되어 있음 보증금은 500만원 A의 주민등록등본상 거주지는 현재 거주지인 원룸이 아닌 친형제 B의 집으로 되어 있음 A가 카드대출을 받아서 다 못 갚고 반환일시금을 받을 수 있는 나이 전에 사망시에 대한 질문입니다 1 A가 다 못 갚은 빚 가족이 갚아야 하는가? 배우자, 부모, 자녀가 없는 현재 상황 따로 상속인을 지정하지 않은 A 형제가 대신 빚을 갚아야 하는가? 형제가 모두 상속포기를 할 경우 조카나 사촌형제에게까지 빚이 넘어가는가? 2 상속포기를 할 경우 형제 4명이 모두 해야 하는가? 상속포기와 한정상속승인이 있던데 재산이 없고 빚만 있는 상황에서도 한정상속이 가능한가? 3 상속포기 혹은 한정상속승인을 할 경우 A의 국민연금 반환일시금을 형제가 받을 수 있는가? A의 모든 재산 및 빚을 상속포기하는 것이니 국민연금 반환일시금도 못 받는 것 아닌가? 아니면 반환일시금을 먼저 형제가 받고 상속포기를 하거나 한정상속승인을 하는 경우가 가능한가? 4 A가 원룸에서 사망시 조카는 보증금을 돌려받을 수 있는가? A의 사인이 자살인 경우 원룸 주인이 손해보는 게 얼만데 보증금을 돌려주냐 그럴 거면 법대로 하자 등의 태도일 경우 조카는 보증금을 돌려받는 게 가능한가? 법적인 절차를 거쳐야 하는가? 소송까지 가지 않고도 받을 수 있는 방법이 있는가? 혹은 소송까지 가지 못하면 조카는 보증금을 돌려받지 못할 가능성이 큰가? 5 A의 사망으로 인해 주민등록등본상 동일 거주지인 형제 B에게 피해는 없는가? 위의 질문에서처럼 상속포기 혹은 한정상속승인 등을 하고 나면 대부업자가 찾아오거나 하는 피해는 없는가? 자세한 답변 부탁드립니다 2번은 절차도",상속
46663,삼촌께서 시한부선고받으시고 재산분배를 하고싶어해요 모르는게많아 질문 남깁니다,"삼촌께선 결혼안하셨고, 부모님,자녀,손주 없으시고 저희아버지 삼촌바로윗형 과 저희어머니,저,여동생 이렇게 18년동안 같이 살고있습니다 등본엔 거주자로 올려져 있구요, 삼촌 명의로 1억1천짜리 아파트 대출5500 , 현금8천 정도를 가지고 계세요 암 말기로 6개월 선고 받으셔서 재산정리를 하려고 하는데, 아파트 대출갚고 저에게 주시고, 나머지를 저희가족들에게 나눠주시려 하고있어요 증여세를 내서라도 저희 가족에게 나눠주고 다른 형제들에게 주기 싫어하고 있어요 사이가 안좋아서 그런데 제가 알기엔 사망시 형제들이 상속1순위로 들어가기 때문에 유류법인가를 신청하면 뺏길수도 있다고 해서 어떻게 해야할지 걱정입니다 유서를 쓰더라도 못피해가는걸로 알고있어서요 재산문제로 싸우는게 싫으시다 하시고 집도 저 결혼할때 주시려고 하셨다고 해서 뺏기는게 싫다고 하십니다 조금이라도 뺏긴다하면 집팔고 돈 다써버리는게 낫다고 하셔요 방법이 있을까요? 명쾌한 답변 부탁드립니다",상속
46664,상속재산 찾고 싶어요,"누군가에게 금전채권을 가지고 있던 아버지가 사망하면서 상속인들 자녀들 의 숫자가 너무 많아 가장 큰 자녀인 형님한테 금전채권추심을 위임하였습니다 이후 큰 형님은 채무자로부터 전부 변제를 받았습니다 액수가 상당히 많습니다 그런데 큰 형님이 나머지 자녀들에게 상속지분대로 돌려주지 않고 미루다가 4년 전에 사망하였고, 큰 형님의 상속인은 아내와 자녀가 있습니다 아버지의 나머지 상속인들이 큰 형님의 상속인인 아내와 자녀들을 상대로 돈을 돌려받고 싶은데, 인터넷이나 주변 법무사에게 문의해 보아도 절차와 방법이 너무 어려운 것 같습니다 액수가 커서 소송을 통해서라도 받고 싶은데, 방법을 모르겠습니다 또 가사법원인지 민사법원인지도 헷갈립니다 너무 어려우면 변호사님을 선임해서 진행할 생각인데 수임료는 어떻게 되나요",상속
46665,재산 상속 대상자 관련 질문,"안녕하세요 재산 상속 관련하여 문의드립니다 할아버지는 돌아가셨구요 할머니의 자손은 아들2, 딸2이 있습니다 한편, 할머니의 큰아들이 이미 사망한 상태입니다 할머니가 사망시 상속자와 상속분이 어떻게 되는지 궁금합니다 큰아들네 상속자가 큰아들의 배우자가 되는 것인지, 큰아들의 큰아들이 되는 것인지, 상속을 받지 못하는 것인지 궁금합니다 답변 부탁드립니다 감사합니다",상속


In [25]:
train_[train_.categoryName == "채권추심"][["_id", "title", "body", "categoryName"]].head()

,_id,title,body,categoryName
98701,59251177fc71765bdaa3a39d,공증받고 빌려준 돈을 받고자 합니다,1년전 돈을 빌려주고 계속 받지 못하여 2017년 4월 경 공증사무소에서 채무자/채권자가 함께 방문하여 공증을 받았습니다 이후 1차례 돈을 받았으나 이후 2차분부터 입금일이 지났으나 돈을 받지 못하고 있어 갖고있는 공증 증서 정본 으로 나머지 금액을 돌려 받고자 합니다 이후 어떤 절차를 통해서 이부분을 해결해야되는건지 궁금합니다 어느정도 시일이 소요되는지? 어느정도 비용이 발생하는지도 궁금합니다,채권추심
98702,5a092183fa0378277d676845,소액 원고료 미지급 민사소송 가능할까요?,네이버 블로그에서 모 업체를 통해 제품 A를 제공받고 블로그 리뷰1건 / 업체 페이지 1건으로 10만원 조금 넘는 금액을 지급 받기로 했는데요 3달넘게 핑계로 지급을 자꾸 미루고있는데 금액이 적은데도 소송이 가능할까요? 모 업체측에서 지급 일을 2번 어기며 자꾸 미루고 있으며 다른 체험건들은 계속 진행중인 상태입니다 소송비용이 받는 금액보다 더 커도 상관이 없으니 업체측을 소송할수 있는지가 더 궁금합니다,채권추심
98707,590606628642097dcade3459,빌려준 돈을 받지 못하고 있습니다,"3년 전에 오백을 빌려줬는데 백만원은 받았고 나머지 금액을 차일피일 미루며 갚아주지 않고 있습니다 저뿐만 아니라 그 채무자에게 돈을 빌려준 사람들이 있는데 같이 묶어서 소송이 가능할까요? 한사람은 삼천, 또 다른사람은 육백 총 사천입니다 그 사람이 다른 사람 돈 오천을 갚지 못해 지금 소송 중인데 그 사람이 처벌을 받는다먼 가중처벌 되나요? 소송이 가능하다면 어떤 방법으로 접근해야 하나요? 그 사람이 갚지 못할시 받는 처벌은 어떻게 되나요?",채권추심
98709,5a2249ffdb5739692f2d3783,친구에게 빌려준돈 얼마를 어디까지 받을수 있을까요,"1 친구의 부탁으로 저는 모르는 제3자에게 이자를 지급받는 조건으로 1000여만원을 빌려주었습니다 이때 돈은 바로 그3자에게 이체하지않고 친구의 계좌를 통해서 이체하였습니다 2 이자는 제3자의 명의 통장으로 저에게 입금되었고, 시간이 지나 그 제3자는 돈을 갚지않고 사기죄로 형을 살게 되었습니다 따라서 저는 돈을 받을수 없는상황에 놓였고 친구에게 대신갚을것을 요구하였습니다 3 이러한 가운데 친구는 투자 목적으로 저에게 6000여만원을 가져갔으며 이에 따른 이득금으로 한3천만원가량을 더주겠다 라는 말로써 저에게 돈을 가져갔습니다 하지만 투자가 잘되지않아 현재 친구는 다른사람과 소송중에 있고 저는 매달 6000만원에 대한 은행이자를 내고 있는 상황에서 친구는 소송이 끝나질 않아 어떠한것도 약속해줄수없다는 입장입니다 위사항 모두 차용증이라던가 보증서 같은것은 작성한적이 없습니다 제가 돈을 위에 천만원과 아래3번의 6천만원 모두 받을수 있나요, 은행이자도 추가적으로 지급받을수 있는지 궁금합니다",채권추심
98710,59a91a675bff4e0fd7522078,동업구두약속 2억투자후 2년동안이익금을 못받음,회사에 재직시 알게된 본인포함4인이 의기투합하여 부동산회사를 설립하여 사업이득금을 n/1로 분해하기로 하고 제가2015년8월경에 2억을 투자하고 동업을 시작하면서 1명이 떨어져 나가고 3명이서 하게됏습니다 사업을 진행하면서 실무를 받고 잇는2명이 어떠한정보도 공개를 하지않고 저는 모든정보를 공유할수잇게해달라요청햇음 지금 현재까지도 공개를 하지않고 사무실이전하고 사무실위치도 알려주지 않습니다 그들은 지금까지도 어떠한 정보도 공개 하지 않으며 죽겟다는 말만 돼풀이 하고 잇으나 기획 부동산 역세권토지분양 특성상 적지않은 이익을 냇을것으로 추정돼나 정보 공개를 전혀 하지 않고 잇어 확인하기 어려움이 잇습니다 동업에 투자한 2억원은 2년동안에 제 급여로 100 400 만원씩을 포함하여 2억 2천여만원을 돌려받앗으나 원래 취지와는 전혀 맞지않고 2년동안 제임금을 포함하여 원금2억으로 계산을 허고 끝내려고 하고 잇습니다 사업이익금은 10억 이상 일것으로 예상하는데 사람만 너무믿는 바람에 분통 터지는 지경에 이르게 돼엇습니다 지금현재도 원활하게 사업을 진행하고 잇으며 시작당시에 자금은 저혼자 2억을 투자 하엿습니다 의리도 저버리고 돈앞에 사람이기를 포기한 저놈들에게 이익금을 환수할 방법이 잇을지 궁금합니다 지금현재도 판매 하고 잇는 토지를 압류하고 법인자산들도 압류 해서 제 정당한몫을 찾을수가 잇을까요?,채권추심


In [16]:
train_[train_.categoryName == "엔터테인먼트"][["title", "body", "categoryName"]].head()

,title,body,categoryName
256,게임 매크로를 사서 사용하면 형사고소의 대상이 될 수 있나요?,"운영진과 개발진이 거의 없다시피하는 게임입니다 게임 내 단속이 없다보니 매크로를 구매해서 사용하는 사람이 무척 많았습니다 듣자하니 매크로를 제작해서 배포하는 경우에는 형사처벌을 받을 수 있다던데, 구매해서 사용한 사람은 형사처벌을 받지 않는 건가요? 그리고 자작한 매크로를 혼자 사용한 경우에는 배포한 경우가 아니므로 이 경우도 형사처벌을 받지 않나요? 그리고 이런 불법프로그램에 관련한 신고는 피해 당사자인 게임사에서만 가능한 것인가요? 아니면 이러한 불법 행위를 목격한 경우 신고를 하면 경찰쪽에서 수사에 들어가게 할 수 있나요?",엔터테인먼트
633,전속계약해지 가능할까요?,"안녕하세요 2019년3월부터 2024년3월까지 공정거래위원회 가수전속계약서를 체결하였는데 회사에서 전속계약을 해놓고 아무활동약속도 지키지 않은 기간이 9개월이 넘어갑니다 8월에 뮤직비디오를 촬영하려 예정했지만 멤버6명중 3명이 나간다고 하여 팀이 엎어졌고 3명에 관해서는 법적으로 위약금소송을 하겠다하였으나 저는 나가겠단 의사를 표시하지 않았고, 카카오톡 단체톡방에서 멤버들을 잡았습니다 근데 회사측에서 그일이 있고 다음날 허락없이 몰래 저희멤버들끼리있는 단톡방 대화를 백업시켰습니다 사생활 보장이란게 있는데 단톡 멤버들끼리의 대화에서 회사윗분들 뒷담화가 많았는데 그글들을 저희를 불러다가 읽게시키려하면서 읽었습니다 근데 대표님 아들이 어리지만 7세? 저희가슴도 만지고 비비는등 추행한 내용이나 자기들 잘못한내용은 다빼버리고 그뒷담화 내용으로 모욕죄니 인성문제를 들먹이시면서 고소하겠다하셨습니다 근데 법적으로 단톡방을 허락없이 백업했는데 그게 더 문제가 되지않나요? 계약서에서도 명시되있는 부분이라 손해배상까지 제가 할수있는걸로 압니다 그 사건이 있은지 4개월이 더 지났고 어떤활동도 하지못하고 회사도 나가지 못하고 계약은 그대로 유지상태입니다 계약건에 대해 얘기하고 싶어도 연락도 받지 않고 그 회사가 대표님 두분이 계신데 한분이 다른 엔터에서 일을 하시고계셔서 사업자가 사라진건지도 모르는 상태고 답답합니다 전속 계약후 데뷔조에서 계약서상으로 당연히 받아야하는 레슨이나 저희의 양성을 위한 부분도 이행이 제대로 되지않았고 스타일리스트가 있다했는데 갑자기 없다면서 저희에게 동대문새벽시장을 돌게하거나, 안무가를 못구해서 뮤비일주일전에 춤을 가르쳐줬다거나 보컬,춤 트레이닝 시켜준다해놓고 트레이너를 몇달째 구하지않고 방치했었고 제 의사와 상관없이 대표님 남편분이 하시는 촬영에 어떤 출연료도 받지못한 채 촬영시킨다던가 하기싫다했음 절 아무것도 못하게 5년동안 계약풀지않고 연락하지않을 생각인거같은데 이 계약 어떻게든 풀수있을까요?",엔터테인먼트
1323,bj 엔터와 계약 종료후 활동 제한 지켜야하나요?,"안녕하세요 최근 bj관련 엔터와 계약을했습니다 계약기간은 1년 내에 93일 방송활동을 조건으로 하고있고 93일이 지나면 계약을 갱신하는 방식입니다 첫 3개월은 5 5수입 분배이구요 제가 여쭤보고 싶은 부분은 계약서에 계약기기간 후, 향후 2년동안 회사에서 매니지먼트 하던 동종 업종에 종사할수없고 만약 방송인이 전단에 규정한 향후 3년이내 동종 업종에 종사할경우 방송인이 위반한 매 일수에 대하여 매 1일에 10만원을 곱한 금액을 손해배상액 예정으로 회사 에게 배상하여야 한다 라고 적혀 있습니다 이부분이 잘 이해가 가지않습니다 2년동안 방송활동을 하지말라는 조항인가요? 지금회사가 제게 지원해준 부분은 캠,책상,컴퓨터,스텐드조명2개 바닥조명1개 2달치 월세 반 부담 정도고 캠등의 물건들은 계약해지후 반납해야하는 조건이에요 반납은 해도상관이없는데 93일 방송일수 채우고 개인으로 방송하고싶거든요 이부분 해결이 가능한지 궁금합니다ㅜㅜ",엔터테인먼트
1403,엔터테이먼트 계약해지,엔터테이먼트와 계약을하고 플랫폼에서 인터넷방송을 했습니다 정신적으로 많이아파져서 정신과도 다니고 약도먹고 하면서 방송을 못하게돼어 엔터와 상의하여 빌려준 장비들을 회수하고 그다음에 계약해지건으로는 대표님과 얘기해야됀다해서 장비를 회수해가지고 대표님께 연락을했는데 받지않는 상황이십니다 이런 계약이 처음인지라 뭔지도 잘모르고 무턱 계약하고나서 정신차리고 보니 계약기간이 48개월로 터무니없이 너무 깁니다 계약서에는 계약해지시 위약금을 얼마를내야한다 이런 정확한 기재는없습니다 계약서쓰러갈때 언제든 하고싶지않으면 나가게해주신다고 하셔서 마음을 놓았는데 이렇게 연락이안돼니 너무너무 심적으로 힘듭니다 계약해지증명서 확인서? 등을 받고싶은데 어떻게해야할까요,엔터테인먼트
1914,엔터테이먼트와 그 소속가수 관련입니다,"저희회사는 중국에서 에이전시업무를 보는회사입니다, 2년정도 전부터 저와 친한 동생이 저희 회사와 계약서는 쓰지않고 중국관련 행사나 sns홍보등 업무를 저희와 계속 보고있었습니다, 그러다가 그 동생이 이번년도 초쯤에 한국의 엔터와 아이돌그룹 전속계약을 하게 되었습니다 맨 처음 그 엔터쪽 대표가 저희와 이 동생이 중국일을 계속 하고있던걸 알고있기에 터치를 하지 않는다 했지만 , 갑자기 개입의 여지를 보이더니 그 동생의 중국진출관련 계약서를 작성하게 되었고 상대방은 그자리에서 도장을 찍었지만 저희는 도장을 찍지 않고 그 서류를 일단 등기로 도장을찍어 보내준다는 말과함께 가지고만 있습니다 이런 상황에서 저희가 그 동생과 같이 계속 중국관련 업무를 보려면 이 엔터와 이 계약을 체결해야 하나요 ?",엔터테인먼트


In [18]:
train_[train_.categoryName == "지역"][["title", "body", "categoryName"]].head()

,title,body,categoryName
536,유일한 공장진입로 차단이 가능할까요?,"1 저희는 96년도 현재 부지 및 공장건물을 경매로 낙찰받아 공장으로 운영중 2 저희 건출물 대장을 확인하여 보니 허가일은 92 01 21 이고, 사용승인일은 92 11 13임 3 옆공장의 건축물은 허가일은 91 08 29이고 사용승인일은 91 01 21 ? 임 4 저희 지번은 옆공장 지번에 91년 분할되었음 5 옆공장 새주인 21 04 매매 이 현재까지 당사 에서 25년이상 사용한 유일한 진입로 아스팔트포장, 폭 4 2m 로 사용중인 도로가 자기네 땅 이므로 펜스를 설치하겠다는 내용증명우편을 발송함 Q1 이경우 진입로 차단이 가능한가요?",지역


In [20]:
df[df.body.str.contains("통매음")].assign(y = lambda x : x.createdAt.dt.year).groupby("y")._id.count()

y
2019    1   
2020    142 
2021    1357
2022    412 
Name: _id, dtype: int64

In [21]:
df[(df.body.str.contains("통매음")) & (df.createdAt.dt.year == 2019)]

,_id,title,title_len,body,body_len,...,category,reject,viewCount,createdAt,categoryName
51500,5db1229024a7d2341248cce4,안녕하세요 통신매체이용음란 관련 문의 합니다,25,통시매체 이용음란 으로 인해 조사를 받기 전에 피해자분이랑 연락이 닿아서 합의를 하게되었고 합의 내용은 처벌을 원하지 않고 고소를 취하한다는 내용이 담긴 합의서를 경찰서에 접수 하였다고 피해자로부터 연락을 받았습니다 제가 궁금한점은 합의서안에 처벌을 원하지 않는 다는 내용이 있으면 별도로 처벌불원서는 제출 안하여도 되는 건가요? 또한 아직 조사전인데 합의서제출이 통매음 조사와 판결에있어서 큰 양형 자료가 될지 걱정입니다,253,...,5a48d537317336d723f0fd51,[],579,2019-10-24 04:03:28.302000+00:00,성범죄


In [23]:
df["y"] = df.createdAt.dt.year

In [25]:
df.groupby(["y", "categoryName"])._id.count().reset_index()

,y,categoryName,_id
0,2015,IT/테크,2
1,2015,건설/부동산,114
2,2015,계약일반,111
3,2015,공정거래,51
4,2015,교통사고/범죄,55
...,...,...,...
430,2022,헌법,2
431,2022,형사기타,158
432,2022,형사소송절차,179
433,2022,환경,4


In [26]:
df.groupby("y").categoryName.nunique()

y
2015    39
2016    40
2017    39
2018    34
2019    69
2020    73
2021    73
2022    68
Name: categoryName, dtype: int64

In [ ]:
tmp